In [1]:
from transformers import pipeline
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import librosa
import torch
import os

/home/lethuan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-01 11:38:05.774935: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-01 11:38:05.783582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 11:38:05.794575: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 11:38:05.798036: E 

In [6]:
folder_path = "/run/user/1000/gvfs/smb-share:server=10.70.115.81,share=ghiam/2024/01/01/"


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = Wav2Vec2Processor.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")
model = Wav2Vec2ForCTC.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")
model.to(device)

def transcribe(wav):
  input_values = processor(wav, sampling_rate=16000, return_tensors="pt").input_values
  logits = model(input_values.to(device)).logits
  pred_ids = torch.argmax(logits, dim=-1)
  pred_transcript = processor.batch_decode(pred_ids)[0]
  return pred_transcript


In [8]:
def resampled_wav_function(wav, orig_sr, target_sr):
  resampled_wav = librosa.resample(wav, orig_sr=orig_sr, target_sr=target_sr)
  return resampled_wav

In [ ]:
# Check if the folder exists
if os.path.exists(folder_path):
    # Loop through each file in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Only process files, not subdirectories
        if os.path.isfile(file_path):
            wav, _ = librosa.load(file_path, sr = 16000)
            print(f"transcript: {transcribe(wav)}")

In [11]:
transcriber = pipeline("automatic-speech-recognition", model="vinai/PhoWhisper-medium")

In [15]:
import sqlite3

In [16]:
con = sqlite3.connect("database.db")
cur = con.cursor()

In [21]:
list_wav = []
for row in cur.execute("SELECT file_name, data FROM wav"):
    list_wav.append(row[1])

In [22]:
list_wav[0]

'[-3.1173933e-02 -5.0760783e-02 -6.5204047e-02 ... -2.7021230e-08\n -4.3535687e-07 -3.6462035e-07]'

In [34]:
import sounddevice as sd


OSError: PortAudio library not found

In [33]:
temp = list_wav[0].strip('')
type(temp)

str

In [14]:

if os.path.exists(folder_path):
# Loop through each file in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # Only process files, not subdirectories
        if os.path.isfile(file_path):
            audio_path = file_path
            wav, sr = librosa.load(audio_path, sr = None)
            if sr != 16000:
                wav = resampled_wav_function(wav, orig_sr=sr, target_sr=16000)

            output = transcriber(wav,return_timestamps=True)['text']

            print(f"Audio {file_name}, sr = {sr}hz, chanel = {wav.shape}: {output}")


/home/lethuan/.local/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during gener

Audio external-7411-+84325556721-20240101-095230-1704077550.450390.wav, sr = 8000hz, chanel = (738240,): dạ dày chín sớm hôm bốn bộ phận chăm sóc khách hàng vi pi xin nghe ờ chị chào em em ơi phải em là bên tổng cục của bến vai phai không dạ em xin lỗi chị cần hỗ trợ gì ạ ờ chị cần di dời á di dời cái qua bên đây rồi qua bên kia gì đó giống như nhà chị di dời quá ấp một qua ấp hai qua ấp ba à dạ rồi chị muốn di dời không phải lộ chị đang ở ấp ba mà di dời quá ấp hai lộ thì bây giờ chị gọi một.tám không một một sáu sáu bấm phím mọ bấm phím ba giùm em nha một tám không một một sáu sáu bấm phím ba chị yêu cầu giùm em ạ các bạn phải tiếp nhận nha dạ dạ rồi em cám ơn.
Audio external-7411-+84368859534-20240101-120714-1704085634.450713.wav, sr = 8000hz, chanel = (0,): đó là sự tấn công cho tội nghiệp thứ hai và đúng tên là đảng cộng.
Audio external-7411-+84906093999-20240101-144636-1704095196.450930.wav, sr = 8000hz, chanel = (0,): đó là sự tấn công cho tội nghiệp thứ hai và đúng tên là đảng 

KeyboardInterrupt: 